In [1]:
from fast_bert.data_abs import BertAbsDataBunch
from box import Box
import logging
import torch
from pathlib import Path

In [2]:
PATH = Path("../../summarisation/")
DATA_PATH = PATH/'data'

In [3]:
args = Box({
    "max_seq_length": 512,
    "batch_size": 8,
    "learning_rate": 5e-3,
    "num_train_epochs": 6,
    "fp16": True,
    "model_name": 'bert-base-uncased',
    "model_type": 'bert'
})

In [4]:
device = torch.device('cuda') if torch.cuda.device_count() else torch.device('cpu')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [5]:
databunch = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=args.model_name, device=device)

In [6]:
databunch.test_dl.dataset[1]

('f7.txt',
 (['customer received a phone call from a supplier she is not familiar with What is the customer complaining about?customer received a phone call from a supplier she is not familiar with What advice have you given the customer? What actions have you taken?the person on the line wanted to hand the call to the supervisor as according to them the customer can safe ¬£500.00 per year on her gas and electric there is no name of the actual supplier or the person on the line, the account is still active and the customer is a bit upset regarding the call. Is this a repeat Complaint? no Has the customer been D+1 signposted?(Have you asked the following question:"Are you aware of the britishgas complaint handling procedure and where to locate it?" If No, "Our Complaint Handling Procedure details the different channels and escalation routes into our business and is available on britishgas.co.uk/energycomplaints or I can post a copy to you free of charge") Have you arranged to call the c

In [7]:
texts = [
    'Customer called through to BG on 27.8.16 about a leak from the shower, an engineer attended from dyno on 31.8.16 (SO-6757449-GDZS). The engineer spent about 5 mins at thr property shone his torch down the side of the shower and blamed the leak on sealent, he offered the customer a temp repair that woulf have been chargeable but the customer declined. The first engineer then left the property promptly without locating a leak and assured the customer that it was to do with the sealent. Since the appt the customer noticed that the leak was still occuring and causing damage and she called for another dyno engineer booked for 10.11.16 (SO-6985967-8DHD). The engineer arrived at the property and pulled the shower out to look behind and found that there were 2 leaks behind the shower that had been ongoing for a long time and he said that if the first engineer would have looked properly then she wouldnt have had so much damage to the property. pics were taken of damage and given to LO and customer is expecting to be compensated for the damaged caused by the leak that was overlook. The LO (SSP LTD - 3439) said that it was consequencial damage that was a result of the fir st leak and she would not be compensated as she declined the temp fix unfortunately the 2nd engineer did not leave sufficient note to suggest that the leak was overlooked in the first instance.',
    'customer received a phone call from a supplier she is not familiar with What is the customer complaining about?customer received a phone call from a supplier she is not familiar with What advice have you given the customer? What actions have you taken?the person on the line wanted to hand the call to the supervisor as according to them the customer can safe ¬£500.00 per year on her gas and electric there is no name of the actual supplier or the person on the line, the account is still active and the customer is a bit upset regarding the call. Is this a repeat Complaint? no Has the customer been D+1 signposted?(Have you asked the following question:"Are you aware of the britishgas complaint handling procedure and where to locate it?" If No, "Our Complaint Handling Procedure details the different channels and escalation routes into our business and is available on britishgas.co.uk/energycomplaints or I can post a copy to you free of charge") Have you arranged to call the customer back? If so, when?no Can I just check/confirm that you are happy for this Complaint to be closed?yes.'
]

In [8]:
databunch_momory = BertAbsDataBunch(data_dir=None, test_data=texts, tokenizer=args.model_name, device=device)

In [9]:
databunch_momory.test_dl.dataset[0]

(None,
 (['Customer called through to BG on 27.8.16 about a leak from the shower, an engineer attended from dyno on 31.8.16 (SO-6757449-GDZS). The engineer spent about 5 mins at thr property shone his torch down the side of the shower and blamed the leak on sealent, he offered the customer a temp repair that woulf have been chargeable but the customer declined. The first engineer then left the property promptly without locating a leak and assured the customer that it was to do with the sealent. Since the appt the customer noticed that the leak was still occuring and causing damage and she called for another dyno engineer booked for 10.11.16 (SO-6985967-8DHD). The engineer arrived at the property and pulled the shower out to look behind and found that there were 2 leaks behind the shower that had been ongoing for a long time and he said that if the first engineer would have looked properly then she wouldnt have had so much damage to the property. pics were taken of damage and given to L